In [1]:
!pip install simpleaudio


  Using cached simpleaudio-1.0.4.tar.gz (2.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for simpleaudio: filename=simpleaudio-1.0.4-cp313-cp313-win_amd64.whl size=2039347 sha256=04551d343b604265a162d90a28319eaf57c8cf676a37f892c6c56a7e2b64fce1
  Stored in directory: c:\users\rawat\appdata\local\pip\cache\wheels\32\52\44\2d31774644c6aedd866574b86a30f6576c6dddc31fa8fb84b3
Successfully built simpleaudio


  DEPRECATION: Building 'simpleaudio' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'simpleaudio'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
import elevenlabs

import os
import base64
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\rawat\anaconda3\envs\shorts\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
!pip install unrealspeech


   ---------------------------------------- 2/2 [unrealspeech]



In [3]:
import os

In [8]:
!pip install simpleaudio

  Using cached simpleaudio-1.0.4.tar.gz (2.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for simpleaudio
Failed to build simpleaudio


  DEPRECATION: Building 'simpleaudio' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'simpleaudio'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [31 lines of output]
      C:\Users\rawat\anaconda3\envs\shorts\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      C:\Users\rawat\anaconda3\envs\shorts\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************

In [3]:
from unrealspeech import UnrealSpeechAPI
import os

In [113]:
speech_api = UnrealSpeechAPI(api_key=os.getenv('UNREAL_SPEACH_API_KEY'))


response = speech_api.speech(
    text='''The rain tapped gently against the window as Arjun sat in the dim café, sketching faces he’d never met.
A woman in a red coat walked in, her eyes scanning the room like she was searching for something… or someone.
She spotted Arjun, smiled faintly, and sat across from him without a word.
He blinked, confused.
“You draw strangers,” she said softly.
“Yes… how do you know?”
“Because you’ve drawn me before,” she whispered, sliding a worn notebook onto the table.
Inside, every page was filled with his sketches—years before they ever met.''',
    voice_id="Will",      # Choose voice: Scarlett, Dan, Liv, etc.
    timestamp_type="word",    # word or sentence
    bitrate="192k"
)


In [114]:
response

{'CreationTime': '2025-08-14T00:56:24.07Z',
 'OutputUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/5e248c8f-e0b0-4881-bcf5-85488ee06580-0.mp3',
 'RequestCharacters': 538,
 'TaskId': '5e248c8f-e0b0-4881-bcf5-85488ee06580',
 'TaskStatus': 'completed',
 'TimestampsUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/5e248c8f-e0b0-4881-bcf5-85488ee06580-0.json',
 'VoiceId': 'Will'}

In [47]:
import requests

In [88]:
response

{'CreationTime': '2025-08-14T00:39:38.59Z',
 'OutputUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/397ffc2e-62da-495e-90a8-7c84f3abbcda-0.mp3',
 'RequestCharacters': 61,
 'TaskId': '397ffc2e-62da-495e-90a8-7c84f3abbcda',
 'TaskStatus': 'completed',
 'TimestampsUri': 'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/397ffc2e-62da-495e-90a8-7c84f3abbcda-0.json',
 'VoiceId': 'Will'}

In [49]:
generated_text=input("Enter a text")

In [50]:
c=requests.get('https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/b9bad842-42ab-4f90-b735-3e16c368d28c-0.json')

In [51]:
c.content

b'[\n    {\n        "start": 0.5,\n        "end": 0.8173393818239371,\n        "word": "Hello",\n        "text_offset": 0\n    },\n    {\n        "start": 0.8173393818239371,\n        "end": 0.9876190523306528,\n        "word": "I",\n        "text_offset": 6\n    },\n    {\n        "start": 0.9876190523306528,\n        "end": 1.1308087743818758,\n        "word": "am",\n        "text_offset": 8\n    },\n    {\n        "start": 1.1308087743818758,\n        "end": 1.4674981062610941,\n        "word": "gay.",\n        "text_offset": 11\n    }\n]'

In [52]:
# from elevenlabs import ElevenLabs

# client=ElevenLabs(api_key=os.getenv('ELEVENLABS_API_KEY'))


# response=client.text_to_speech.convert_with_timestamps(
#     voice_id="NFG5qt843uXKj4pFvR7C",
#     text=generated_text
# )

In [94]:
sound=requests.get(response['OutputUri'])

In [54]:
response['TimestampsUri']

'https://unreal-expire-in-90-days.s3-us-west-2.amazonaws.com/aa5a3a7b-8c7b-4e52-82db-159dc883b8a7-0.json'

In [ ]:
timestampurl=requests.get(response['TimestampsUri'])

In [64]:
timestampurl

<Response [200]>

In [96]:
import json


In [115]:
with open('timestamps.json','wb') as file:
    file.write(timestampurl.content)

In [ ]:
with open('goon.mp3','wb') as file:
    file.write(sound.content)

In [60]:
from unrealspeech import UnrealSpeechAPI
speech_api = UnrealSpeechAPI(api_key=os.getenv('UNREAL_SPEACH_API_KEY'))

r1 = speech_api.speech(
    text=generated_text,
    voice_id="Scarlett",  # or Dan, Amy, Liv, Will
    timestamp_type="word",  # or "sentence"
    bitrate="192k",
    speed=0.0,
    pitch=1.0
)


HTTPError: 400 Client Error: Bad Request for url: https://api.v6.unrealspeech.com/speech

In [ ]:
# audio=base64.b64decode(response.audio_base_64)

# with open('prime.mp3','wb') as file:
#     file.write(audio)

In [116]:
c=timestampurl.content

In [102]:
import json

In [117]:
c

b'[\n    {\n        "start": 0.5,\n        "end": 0.5735298569003741,\n        "word": "The",\n        "text_offset": 0\n    },\n    {\n        "start": 0.5735298569003741,\n        "end": 0.8869992457330227,\n        "word": "rain",\n        "text_offset": 4\n    },\n    {\n        "start": 0.8869992457330227,\n        "end": 1.3126984151701133,\n        "word": "tapped",\n        "text_offset": 9\n    },\n    {\n        "start": 1.3126984151701133,\n        "end": 1.7306576011081536,\n        "word": "gently",\n        "text_offset": 16\n    },\n    {\n        "start": 1.7306576011081536,\n        "end": 2.117656848082939,\n        "word": "against",\n        "text_offset": 23\n    },\n    {\n        "start": 2.117656848082939,\n        "end": 2.210536664972702,\n        "word": "the",\n        "text_offset": 31\n    },\n    {\n        "start": 2.210536664972702,\n        "end": 2.6749357581138606,\n        "word": "window",\n        "text_offset": 35\n    },\n    {\n        "start":

In [118]:
data_str = c.decode("utf-8")  # bytes → string
data_json = json.loads(data_str) 

In [119]:
data_json

[{'start': 0.5, 'end': 0.5735298569003741, 'word': 'The', 'text_offset': 0},
 {'start': 0.5735298569003741,
  'end': 0.8869992457330227,
  'word': 'rain',
  'text_offset': 4},
 {'start': 0.8869992457330227,
  'end': 1.3126984151701133,
  'word': 'tapped',
  'text_offset': 9},
 {'start': 1.3126984151701133,
  'end': 1.7306576011081536,
  'word': 'gently',
  'text_offset': 16},
 {'start': 1.7306576011081536,
  'end': 2.117656848082939,
  'word': 'against',
  'text_offset': 23},
 {'start': 2.117656848082939,
  'end': 2.210536664972702,
  'word': 'the',
  'text_offset': 31},
 {'start': 2.210536664972702,
  'end': 2.6749357581138606,
  'word': 'window',
  'text_offset': 35},
 {'start': 2.6749357581138606,
  'end': 3.1006349076827364,
  'word': 'as',
  'text_offset': 42},
 {'start': 3.1006349076827364,
  'end': 3.5418140453596907,
  'word': 'Arjun',
  'text_offset': 45},
 {'start': 3.5418140453596907,
  'end': 3.8978533421953516,
  'word': 'sat',
  'text_offset': 51},
 {'start': 3.8978533421

In [ ]:
alignment = response.alignment
chars = alignment.characters
starts = alignment.character_start_times_seconds
ends = alignment.character_end_times_seconds

In [ ]:
[{'start': 0.5, 'end': 0.8173393818239371, 'word': 'Hello', 'text_offset': 0},
 {'start': 0.8173393818239371,
  'end': 0.9876190523306528,
  'word': 'I',
  'text_offset': 6},
 {'start': 0.9876190523306528,
  'end': 1.1308087743818758,
  'word': 'am',
  'text_offset': 8},
 {'start': 1.1308087743818758,
  'end': 1.4674981062610941,
  'word': 'gay.',
  'text_offset': 11}]

In [ ]:
data_json[0]['']

{'start': 0.5, 'end': 0.8173393818239371, 'word': 'Hello', 'text_offset': 0}

In [ ]:
# def func(response):
#     alignment = response.alignment
#     chars = alignment.characters
#     starts = alignment.character_start_times_seconds
#     ends = alignment.character_end_times_seconds
#     words = []
#     current_word = ""
#     current_start = None

#     for c, s, e in zip(chars, starts, ends):
#         if c != " ":
#             if current_word == "":  # starting a new word
#                 current_start = s
#             current_word += c
#             current_end = e
#         else:
#             if current_word:  # end of a word
#                 words.append((current_word, current_start, current_end))
#                 current_word = ""

#     # Add the last word if there is one
#     if current_word:
#         words.append((current_word, current_start, current_end))

#     # Print result
#     for word, s, e in words:
#         print(f"'{word}' spoken from {s:.2f}s to {e:.2f}s")


#     return words


In [ ]:
data_json

[{'start': 0.5, 'end': 0.8173393818239371, 'word': 'Hello', 'text_offset': 0},
 {'start': 0.8173393818239371,
  'end': 0.9876190523306528,
  'word': 'I',
  'text_offset': 6},
 {'start': 0.9876190523306528,
  'end': 1.1308087743818758,
  'word': 'am',
  'text_offset': 8},
 {'start': 1.1308087743818758,
  'end': 1.4674981062610941,
  'word': 'gay.',
  'text_offset': 11}]

In [ ]:
# def group_words_into_phrases(words, max_words_per_phrase=3):
#     """Groups individual words into phrases based on punctuation or max word count."""
#     phrases = []
#     current_phrase = []
#     if not words:
#         return []

#     phrase_start_time = words[0][1]

#     for i, (word, start, end) in enumerate(words):
#         current_phrase.append(word)
#         should_end_phrase = any(c in word for c in ",.?!") or (len(current_phrase) >= max_words_per_phrase or i == len(words) - 1)

#         if should_end_phrase:
#             phrase_text = " ".join(current_phrase)
#             phrases.append((phrase_text, phrase_start_time, end))
#             current_phrase = []
#             if i < len(words) - 1:
#                 phrase_start_time = words[i+1][1]

#     return phrases

In [106]:
def group_words_into_phrases(words, max_words_per_phrase=3):
    """Groups individual words (dict format) into phrases based on punctuation or max word count."""
    phrases = []
    current_phrase = []
    if not words:
        return []

    phrase_start_time = words[0]["start"]

    for i, w in enumerate(words):
        word = w["word"]
        start = w["start"]
        end = w["end"]

        current_phrase.append(word)
        should_end_phrase = (
            any(c in word for c in ",.?!")
            or len(current_phrase) >= max_words_per_phrase
            or i == len(words) - 1
        )

        if should_end_phrase:
            phrase_text = " ".join(current_phrase)
            phrases.append({
                "phrase": phrase_text,
                "start": phrase_start_time,
                "end": end
            })
            current_phrase = []
            if i < len(words) - 1:
                phrase_start_time = words[i + 1]["start"]

    return phrases


In [33]:
timestamp=func(response)

'Skibidi' spoken from 0.00s to 0.61s
'bop,' spoken from 0.67s to 1.14s
'yes' spoken from 1.30s to 1.60s
'yes' spoken from 1.71s to 2.01s
'yes!' spoken from 2.11s to 2.67s
'I' spoken from 2.93s to 2.98s
'woke' spoken from 3.09s to 3.35s
'up' spoken from 3.39s to 3.52s
'on' spoken from 3.58s to 3.66s
'a' spoken from 3.70s to 3.73s
'rainbow' spoken from 3.79s to 4.18s
'trampoline' spoken from 4.24s to 4.98s
'made' spoken from 5.12s to 5.38s
'of' spoken from 5.41s to 5.47s
'pickles.' spoken from 5.53s to 6.26s
'My' spoken from 6.99s to 7.12s
'alarm' spoken from 7.17s to 7.45s
'clock' spoken from 7.51s to 7.78s
'was' spoken from 7.85s to 7.98s
'singing' spoken from 8.02s to 8.37s
'jazz' spoken from 8.47s to 8.88s
'in' spoken from 8.99s to 9.11s
'Morse' spoken from 9.17s to 9.47s
'code.' spoken from 9.53s to 10.05s
'Tung-tung!' spoken from 10.22s to 10.95s
'My' spoken from 11.24s to 11.37s
'socks' spoken from 11.45s to 11.81s
'decided' spoken from 11.87s to 12.46s
'they' spoken from 12.52s t

In [107]:
phrases=group_words_into_phrases(data_json)

In [73]:
phrases

[{'phrase': 'Hello I am', 'start': 0.5, 'end': 1.328178368508816},
 {'phrase': 'walter hardwill white',
  'start': 1.328178368508816,
  'end': 2.7136356544991336},
 {'phrase': 'i live in',
  'start': 2.7136356544991336,
  'end': 3.212864683320125},
 {'phrase': 'panama new mexico.',
  'start': 3.212864683320125,
  'end': 4.435782289753357}]

In [120]:
import pygame
pygame.mixer.init()
pygame.mixer.music.load("ii.mp3")

In [109]:
import ffmpeg

def add_timed_text(stream, text, start, end, fontsize_ratio=0.12, fontcolor="white", outline=6):
    """
    Adds large, bold, centered YouTube Shorts-style text.
    """
    return stream.drawtext(
        text=text,
        fontfile=r"C:\Windows\Fonts\impact.ttf",   # Bold meme/viral look
        fontsize=f"h*{fontsize_ratio}",            # Scale with video height
        fontcolor=fontcolor,
        x="(w-text_w)/2",                          # Center horizontally
        y="(h-text_h)/2",                          # Center vertically
        borderw=outline,                           # Thick outline
        bordercolor="black",
        box=0,                                     # No box, outline only
        enable=f"between(t,{start},{end})"
    )


In [110]:
def overlay(stream,timestamp):
   '''
   Function: Apply timestamp

   Args:
      stream: Canvas to apply work
      timestamp: Words with start and end time
   '''
   for word,start,end in timestamp:
       stream=add_timed_text(stream,word,start,end)
    
   return stream
    


In [111]:
import ffmpeg

input_file = "video.mp4"
audio_file = "goon.mp3"
output_file = "asscream.mp4"  #mp4 to apply subtitles to

# Load video
video = ffmpeg.input(input_file).video
audio = ffmpeg.input(audio_file).audio

video=overlay(video,phrases)

(
    ffmpeg
    .output(video, audio, output_file, vcodec='libx264', acodec='aac', strict='experimental')
    .overwrite_output()
    .run()
)


Error: ffmpeg error (see stderr output for detail)

In [121]:
import ffmpeg

def add_timed_text(stream, text, start, end, fontsize_ratio=0.12, fontcolor="white", outline=6):
    """
    Adds large, bold, centered YouTube Shorts-style text.
    """
    return stream.drawtext(
        text=text,
        fontfile=r"C:\Windows\Fonts\impact.ttf",   # Bold meme/viral look
        fontsize=f"h*{fontsize_ratio}",            # Scale with video height
        fontcolor=fontcolor,
        x="(w-text_w)/2",                          # Center horizontally
        y="(h-text_h)/2",                          # Center vertically
        borderw=outline,                           # Thick outline
        bordercolor="black",
        box=0,                                     # No box, outline only
        enable=f"between(t,{start},{end})"
    )

def overlay(stream, phrases):
    """
    Apply timed phrases to the video stream.
    phrases: list of dicts with keys 'phrase', 'start', 'end'
    """
    for p in phrases:
        stream = add_timed_text(stream, p["phrase"], p["start"], p["end"])
    return stream

# Example usage
input_file = "video.mp4"
audio_file = "ii.mp3"
output_file = "asscream.mp4"

video = ffmpeg.input(input_file).video
audio = ffmpeg.input(audio_file).audio

# phrases = [{'phrase': 'Hello I am gay.', 'start': 0.5, 'end': 1.46}, ...]
video = overlay(video, phrases)

(
    ffmpeg
    .output(video, audio, output_file, vcodec='libx264', acodec='aac', strict='experimental')
    .overwrite_output()
    .run()
)


(None, None)